# An example: running BuDRO on the German credit data set

In [22]:
import time

In [3]:
import numpy as np
import scipy as sp
import tensorflow as tf
import xgboost as xgb

In [2]:
## load the BuDRO scripts
import sys
sys.path.append('../scripts')

Import BuDRO scripts.  `geman/german_proc.py` contains functions for loading and processing the German credit data set.  `scripts/script.py` contains various helper functions for collecting statistics.  `scripts/tf_xgboost_log.py` contains all of the boosting functions for BuDRO (primal vs dual, SGD vs without SGD).

In [4]:
import german_proc
import script
import tf_xgboost_log as txl

In [5]:
# define dtype for a default.  
_dtype = tf.float32
_verbose = False

## Load seeds and get a train/test split

In [6]:
seeds = np.load('german-seeds.npz')['seeds']

In [7]:
seed = seeds[0]

In [8]:
orig_dataset = german_proc.get_german_data()

In [9]:
(
    X_train,
    X_test,
    y_train,
    y_test,
    y_age_train,
    y_age_test,
    feature_names,
    traininds,
    testinds
) = german_proc.get_german_train_test_age(
        orig_dataset,
        pct=0.8,    # percent data in training set
        removeProt=False,    # remove or keep protected features
        seed=seed
)

[0, 1, 2, 3, 4, 5, 6]


Indices of protected features and status features for running consistency evaluations.  Also create a binary age feature.

In [12]:
aind = feature_names.index('age')
abind = feature_names.index('age_bin')

status_inds = []
y_sex_bin = np.zeros(X_train.shape[0])
yt_sex = np.zeros(X_test.shape[0])

for i, nom in enumerate(feature_names):
    if "personal_status" in nom:
        status_inds.append(i)
        
        # Males are 1, females are 0 in the following 
        if 'A92' not in nom and 'A95' not in nom:
            y_sex_bin  += X_train[:,i]
            yt_sex += X_test[:,i]

print("Personal status indices: {}".format(status_inds))

Personal status indices: [37, 38, 39, 40]


In [13]:
feature_names[37]

'personal_status=A91'

Get the binary age feature and remove it from the data.  See the note about agebin in the `00_README.txt` file.

In [14]:
y_age_bin = np.copy(X_train[:, abind])
yt_age = np.copy(X_test[:, abind])

X_train[:, abind] = 0
X_test[:, abind] = 0 

## Set up the BuDRO parameters
Distance matrix, etc

In [15]:
# first project the data onto the sensitive subspace, then calculate the pairwise distances
RCV = german_proc.train_ridge(X_train, y_age_train, aind=aind, abind=None)
proj = german_proc.german_proj_mat(RCV, aind)
projData = np.matmul(X_train, proj)

Shape of projection matrix: (62, 62)
Projection error: 1.1102230246251565e-16


In [16]:
Corig = sp.spatial.distance.squareform(
        sp.spatial.distance.pdist(
            projData,
            metric='sqeuclidean'
        )
)

C = tf.constant(Corig, dtype=_dtype)

In [18]:
n = Corig.shape[0]

In [19]:
# XGBoost data objects and tree parameters
dtrain, dtest, watchlist, param, _, _ = german_proc.prep_baseline_xgb(X_train, X_test, y_train, y_test)

#### Tweak the values in param to change the input hyperparameters.

Generally keep `scale_pos_weight` untouched, and don't touch the objective.  Otherwise, I've found good results by keeping the tree pretty short and regularizing quite a bit (`lambda` on the order of 1 - 1000, fairly large `min_child_weight` 0.1-10)

In [20]:
param

{'max_depth': 5,
 'eta': 0.05,
 'objective': 'binary:logistic',
 'min_child_weight': 0.5,
 'lambda': 0.0001,
 'scale_pos_weight': 2.3333333333333335}

## Run BuDRO to reproduce one of the hand-tuning runs

In [24]:
param['max_depth'] = 10
param['eta']= 0.005 
param['min_child_weight'] = 0.1/n
param['lambda'] = 1.0

n_iter = 500
eps = 1.0

Boosting functions:

* `txl.boost_dual` uses dual without sgd
* `txl.boost_dual_sgd` uses dual with sgd
* `txl.boost_sinkhorn_tf` uses entropic regularization without sgd
* `txl.boost_sinkhorn_2gpu` uses entropic regularization without sgd explicitly on 2 gpus.  This code is not portable and should be avoided if possible. 
* `txl.boost_sinkhorn_sgd_2gpu` uses entropric regularization with sgd and expects 2 gpus. 

Please see the file `scripts/tf_xgboost_log.py` for the function arguments and implementations.

In the output for the following, we are minimizing the inner max - thus, that quantity should be decreasing.  If it's not, mess around with the parameters some more.

In [25]:
t_s = time.time()
use_dual = True
# fst is the trained GBDT object, xport is the transport map in pairs.  stuff is extra information about a
# solution that is ambiguous (e.g. a point is spread out over multiple outputs)
fst, xport, stuff = txl.boost_dual(
    X_train,
    y_train,
    C,
    n_iter,
    X_test=X_test,
    y_test=y_test,
    pred=None,
    eps=eps,       # perturbation budget
    param=param,   # parameters, above
    verbose=_verbose,
    verify=True,
    lowg=0.0,      # lower and upper guesses for the bisection method.  A smaller range will result in 
    highg=15.0,    # a shorter runtime, but the optimal eta could be missed.  See Appendix B.3.
    method='brentq',
    dtype=_dtype,
    outfunc=None,   # how to save the data to a tensorboard file, see `german_fair_age.py`
)

print("Boosting took %f" % (time.time() - t_s))

Iter 0: Final value of inner max: 0.6914510862529278
Iter 1: Final value of inner max: 0.6913871238729493
Iter 2: Final value of inner max: 0.6913456810265779
Iter 3: Final value of inner max: 0.6913054555654525
Iter 4: Final value of inner max: 0.6912731740868736
Iter 5: Final value of inner max: 0.6912468572827626
Iter 6: Final value of inner max: 0.6912159740139269
Iter 7: Final value of inner max: 0.6911799963616841
Iter 8: Final value of inner max: 0.69113916606486
Iter 9: Final value of inner max: 0.6911111467331648
Iter 10: Final value of inner max: 0.6910489483177662
Iter 11: Final value of inner max: 0.6910253007802201
Iter 12: Final value of inner max: 0.6909951419532356
Iter 13: Final value of inner max: 0.6909613249450922
Iter 14: Final value of inner max: 0.6909336142987013
Iter 15: Final value of inner max: 0.6908779411017896
Iter 16: Final value of inner max: 0.6908568576371931
Iter 17: Final value of inner max: 0.6908369352668524
Iter 18: Final value of inner max: 0.690

Iter 141: Final value of inner max: 0.6873617044836282
Iter 142: Final value of inner max: 0.6873367621004581
Iter 143: Final value of inner max: 0.6873167233335615
Iter 144: Final value of inner max: 0.6872963924834541
Iter 145: Final value of inner max: 0.6872841471051725
Iter 146: Final value of inner max: 0.6872377872945277
Iter 147: Final value of inner max: 0.6872268605205842
Iter 148: Final value of inner max: 0.6872155416756869
Iter 149: Final value of inner max: 0.6872064643353224
Iter 150: Final value of inner max: 0.6871736271679402
Iter 151: Final value of inner max: 0.6871471609175206
Iter 152: Final value of inner max: 0.6870865130424499
Iter 153: Final value of inner max: 0.6870686276197586
Iter 154: Final value of inner max: 0.6870512844135968
Iter 155: Final value of inner max: 0.6870311967478221
Iter 156: Final value of inner max: 0.6870069048553705
Iter 157: Final value of inner max: 0.6869708538723616
Iter 158: Final value of inner max: 0.6869342570841099
Iter 159: 

Iter 287: Final value of inner max: 0.683701735822728
Iter 288: Final value of inner max: 0.6836833859980107
Iter 289: Final value of inner max: 0.683678777590394
Iter 290: Final value of inner max: 0.6836590974032879
Iter 291: Final value of inner max: 0.6835922434865724
Iter 292: Final value of inner max: 0.6835798062666161
Iter 293: Final value of inner max: 0.6835450796041775
Iter 294: Final value of inner max: 0.6835146425664425
Iter 295: Final value of inner max: 0.6834697360545396
Iter 296: Final value of inner max: 0.6834826292097569
Iter 297: Final value of inner max: 0.6834393246474078
Iter 298: Final value of inner max: 0.6834219286069811
Iter 299: Final value of inner max: 0.6834126716198564
Iter 300: Final value of inner max: 0.6833976822183332
Iter 301: Final value of inner max: 0.6833754934370518
Iter 302: Final value of inner max: 0.6833506166189909
Iter 303: Final value of inner max: 0.683332399433479
Iter 304: Final value of inner max: 0.683298641363036
Iter 305: Fina

Iter 437: Final value of inner max: 0.6802034498803455
Iter 438: Final value of inner max: 0.680212764069438
Iter 439: Final value of inner max: 0.6801758870482445
Iter 440: Final value of inner max: 0.6801415503107421
Iter 441: Final value of inner max: 0.6801134432493906
Iter 442: Final value of inner max: 0.6801000665256396
Iter 443: Final value of inner max: 0.680065809735424
Iter 444: Final value of inner max: 0.6800542785240148
Iter 445: Final value of inner max: 0.6799929788708687
Iter 446: Final value of inner max: 0.679985922202468
Iter 447: Final value of inner max: 0.6799692156165839
Iter 448: Final value of inner max: 0.6799416894612854
Iter 449: Final value of inner max: 0.6799261550076536
Iter 450: Final value of inner max: 0.6799091942802048
Iter 451: Final value of inner max: 0.6798969122767449
Iter 452: Final value of inner max: 0.6798791015893222
Iter 453: Final value of inner max: 0.6798612185567618
Iter 454: Final value of inner max: 0.6798561360687018
Iter 455: Fin

#### Create Pi from the boosting solution

In [29]:
final = 0
if use_dual:
    dists = Corig[xport, np.arange(Corig.shape[1])]
    
    # calculate the exact solution to the linear system with two equations and two unknowns.
    if stuff is not None:
        dists[stuff[0]] = 0
        final += Corig[stuff[0], stuff[2]]*stuff[3] + Corig[stuff[0], stuff[1]]*(1/n - stuff[3])
    
    final += dists.sum()/n
                       
# should be equal to eps
print("Constraint value: {}".format(final))

Constraint value: 0.9999999995500375


## Evaluate the quality of the fair GBDT model

In [30]:
# use the trained model to classify the test set
preds = fst.predict(dtest)
y_guess = np.array([1 if pval > 0.5 else 0 for pval in preds])

In [34]:
# accuracy and group fairness metrics
p0, p1, _ = script.balanced_accuracy(fst, dtest, y_test)
p0t, p1t, _ = script.balanced_accuracy(fst, dtrain, y_train)
age_res = script.group_metrics(
    y_test,
    y_guess,
    yt_age,
    label_good=1,
    verbose=True
)

Accuracy is 0.735000
Protected class:
TPR: 0.7222222222222222; TNR: 0.631578947368421
Priv class:
TPR: 0.7142857142857143; TNR: 0.7603305785123967
Gap RMS: 0.09121395364871711, Gap MAX: 0.1287516311439757
Average odds difference is 0.068344
Equal opportunity difference is 0.007937
Statistical parity difference is 0.178577


In [37]:
print("train balanced: {}".format(.5 * (p0t + p1t)))
print("test balanced: {}".format(.5 * (p0 + p1)))

train balanced: 0.8607142857142858
test balanced: 0.7297619047619048


In [38]:
# status consistency
scons = german_proc.status_cons(fst, X_test, status_inds)

print("status consistency: {}".format(scons.sum()/X_test.shape[0]))

status consistency: 0.96
